In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [2]:
property = pd.read_csv('../Data/original/nyc_property_valuation_2025.csv')

C:\Users\kdgki\AppData\Local\Temp\ipykernel_2676\1778808208.py:1: DtypeWarning: Columns (0,4,5,8,9,22,33,44,55,66,69,70,73,74,75,77,78,82,85,87,92,99,100,104,108,110,115,117,132) have mixed types. Specify dtype option on import or set low_memory=False.
  property = pd.read_csv('../Data/original/nyc_property_valuation_2025.csv')


In [3]:
property["BBL"] = (
    property["BORO"].astype(str)
    + property["BLOCK"].astype(str).str.zfill(5)
    + property["LOT"].astype(str).str.zfill(4)
)

In [4]:
property_sorted = property[['BBL', 'CURTAXCLASS', 'CURMKTTOT', 'PYMKTTOT', 'GROSS_SQFT', 'RESIDENTIAL_AREA_GROSS', 'BLD_STORY', 'ZONING', 'BLDG_CLASS']]

In [5]:
property_sorted["CURMKTTOT"] = pd.to_numeric(property_sorted["CURMKTTOT"], errors='coerce')
property_sorted["CURTAXCLASS"] = pd.to_numeric(property_sorted["CURTAXCLASS"], errors='coerce')
property_sorted["PYMKTTOT"] = pd.to_numeric(property_sorted["PYMKTTOT"], errors='coerce')
property_sorted["GROSS_SQFT"] = property_sorted["GROSS_SQFT"].replace(0, np.nan)
property_sorted["GROSS_SQFT"] = pd.to_numeric(property_sorted["GROSS_SQFT"], errors='coerce')
property_sorted.loc[property_sorted["GROSS_SQFT"] <= 10, "GROSS_SQFT"] = np.nan
property_sorted["price_per_sqft"] = property_sorted["CURMKTTOT"] / property_sorted["GROSS_SQFT"]
property_sorted["price_per_sqft_2024"] = property_sorted["PYMKTTOT"] / property_sorted["GROSS_SQFT"]

C:\Users\kdgki\AppData\Local\Temp\ipykernel_2676\1609957628.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  property_sorted["CURMKTTOT"] = pd.to_numeric(property_sorted["CURMKTTOT"], errors='coerce')
C:\Users\kdgki\AppData\Local\Temp\ipykernel_2676\1609957628.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  property_sorted["CURTAXCLASS"] = pd.to_numeric(property_sorted["CURTAXCLASS"], errors='coerce')
C:\Users\kdgki\AppData\Local\Temp\ipykernel_2676\1609957628.py:3: SettingWithCopyWarning: 
A value i

In [6]:
# property_sorted.to_csv('../Data/processed/property_value_lot_2025.csv', index=False)

In [25]:
file1 = r'../Data/original/nyc_building.geojson'
building_original = gpd.read_file(file1)
manhattan_original = building_original[building_original['mappluto_bbl'].astype(str).str.startswith('1')]
manhattan_original = manhattan_original.sort_values(by='bin', ascending=True)

In [8]:
file3 = r'../Data/processed/station_time_taken.csv'
time_taken_temp = pd.read_csv(file3, header=None, names=['time_taken'])
# order ot time_taken matches with the order of manhattan_original

In [9]:
file4 = r'../Data/original/elevator_permit.csv'
elevator = pd.read_csv(file4)
elevator = elevator[['BIN', 'Elevator Device Type']]
elevator = elevator.rename(columns={'BIN':'bin', 'Elevator Device Type':'elevator'})

C:\Users\kdgki\AppData\Local\Temp\ipykernel_2676\3674851643.py:2: DtypeWarning: Columns (11,20,23,24,36,56) have mixed types. Specify dtype option on import or set low_memory=False.
  elevator = pd.read_csv(file4)


In [10]:
pluto = pd.read_csv('../Data/original/Primary_Land_Use_Tax_Lot_Output__PLUTO__20250822.csv')

C:\Users\kdgki\AppData\Local\Temp\ipykernel_2676\2204848067.py:1: DtypeWarning: Columns (17,18,20,21,22,24,25,52,61,62,79,97) have mixed types. Specify dtype option on import or set low_memory=False.
  pluto = pd.read_csv('../Data/original/Primary_Land_Use_Tax_Lot_Output__PLUTO__20250822.csv')


In [11]:
property_sorted = property_sorted.rename(columns={'BBL': 'base_bbl'})
print(property_sorted.columns)

Index(['base_bbl', 'CURTAXCLASS', 'CURMKTTOT', 'PYMKTTOT', 'GROSS_SQFT',
       'RESIDENTIAL_AREA_GROSS', 'BLD_STORY', 'ZONING', 'BLDG_CLASS',
       'price_per_sqft', 'price_per_sqft_2024'],
      dtype='object')


In [12]:
bin_temp = pd.read_csv('../Data/processed/bin_for_time_taken.csv', header=None)
bin_temp.columns = ['bin']

In [13]:
time_taken = pd.concat([bin_temp, time_taken_temp], axis = 1)
time_taken['bin'] = pd.to_numeric(time_taken['bin'], errors = 'coerce')
time_taken = time_taken.sort_values(by = 'bin', ascending = True).reset_index(drop=True)
time_taken.head()

,bin,time_taken
0,1000000,12.173757
1,1000000,2.727661
2,1000002,20.000000
3,1000003,1.629435
4,1000004,1.419374


In [14]:
pluto['base_bbl'] = (
    pluto['borocode'].astype(str)
    +pluto['Tax block'].astype(str).str.zfill(5)
    +pluto['Tax lot'].astype(str).str.zfill(4)
)

In [ ]:
res = pluto[['base_bbl', 'unitsres']]
res['base_bbl'] = pd.to_numeric(res['base_bbl'], errors='coerce').astype('Int64')
manhattan = manhattan_original[['base_bbl', 'construction_year', 'height_roof', 'bin', 'geometry']].reset_index(drop=True)

manhattan['bin'] = manhattan['bin'].astype('Int64')
manhattan['base_bbl'] = pd.to_numeric(manhattan['base_bbl'], errors='coerce').astype('Int64')
property_sorted['base_bbl'] = pd.to_numeric(property_sorted['base_bbl'], errors='coerce').astype('Int64')

manhattan = manhattan.dropna(subset=['bin']).copy().reset_index(drop=True)
time_taken = time_taken.dropna(subset=['bin']).drop_duplicates(subset=['bin'], keep='first').reset_index(drop=True)

manhattan = manhattan.merge(property_sorted, on="base_bbl", how="left")
manhattan = manhattan.merge(time_taken, on='bin', how='left')
manhattan = manhattan.merge(elevator, on='bin', how='left')
manhattan = manhattan.merge(res, on='base_bbl', how='left')

C:\Users\kdgki\AppData\Local\Temp\ipykernel_2676\1788520427.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res['base_bbl'] = pd.to_numeric(res['base_bbl'], errors='coerce').astype('Int64')


In [ ]:
manhattan = manhattan.set_geometry('geometry', drop=False)
manhattan['base_bbl'] = pd.to_numeric(manhattan['base_bbl'], errors='coerce')
manhattan['time_taken'] = pd.to_numeric(manhattan['time_taken'], errors='coerce')
manhattan['construction_year'] = pd.to_numeric(manhattan['construction_year'], errors='coerce')
manhattan['height_roof'] = pd.to_numeric(manhattan['height_roof'], errors='coerce')

C:\Users\kdgki\AppData\Local\Temp\ipykernel_2676\3507274611.py:1: FutureWarning: The `drop` keyword argument is deprecated and in future the only supported behaviour will match drop=False. To silence this warning and adopt the future behaviour, stop providing `drop` as a keyword to `set_geometry`. To replicate the `drop=True` behaviour you should update your code to
`geo_col_name = gdf.active_geometry_name; gdf.set_geometry(new_geo_col).drop(columns=geo_col_name).rename_geometry(geo_col_name)`.
  manhattan = manhattan.set_geometry('geometry', drop=False)


In [ ]:
bbl_counts = manhattan.groupby("base_bbl")["bin"].transform("count")

# columns to divide
cols_to_adjust = ['CURMKTTOT', 'PYMKTTOT', 'GROSS_SQFT', 'RESIDENTIAL_AREA_GROSS', 'unitsres']

# divide each column by the per-BBL count
for col in cols_to_adjust:
    if col in manhattan.columns:
        manhattan[col] = manhattan[col] / bbl_counts


In [52]:

manhattan['unitsres'].sum()

np.float64(768051.9999999999)

In [48]:
# top lots by unitsres
manhattan[['base_bbl','unitsres']].dropna().sort_values('unitsres', ascending=False).head(10)

# check one suspicious lot
bbl0 = manhattan.loc[manhattan['unitsres'].idxmax(), 'base_bbl']
manhattan.loc[manhattan['base_bbl']==bbl0, ['base_bbl','bin','unitsres','CURTAXCLASS','BLDG_CLASS', 'geometry']].drop_duplicates().head(20)


,base_bbl,bin,unitsres,CURTAXCLASS,BLDG_CLASS,geometry
220316,1014260044,1043955,106.875,2.0,D6,"MULTIPOLYGON (((-73.96013 40.7694, -73.96016 4..."


In [49]:
manhattan.to_file(r'../Data/processed/manhattan_buildings.geojson', driver='GeoJSON')